In [1]:
import requests
import random
import re
import pronouncing as pr


In [2]:


# Define the URL to fetch poems
url = "https://poetrydb.org/author,title/Shakespeare;Sonnet"

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    poems = response.json()
    poem_lines = [line for poem in poems for line in poem['lines']]
else:
    print("Failed to fetch poems.")

def count_syllables(text):
    # Remove non-alphabetic characters
    text = re.sub('[^a-z]+', ' ', text.lower())
    phones = [pr.phones_for_word(p)[0] for p in text.split() if pr.phones_for_word(p)]
    syllable_count = sum([pr.syllable_count(p) for p in phones])

    return syllable_count


def preprocess_verse(verse):
    """Lowercase and filter all punctuation"""
    # Lowercase
    verse_lower = verse.strip().lower()
    # Filter punctuation (keep numbers)
    return re.sub('[^a-z \d]+', '', verse_lower)

def last_word(verse):
    """Retrieve the last word of a verse"""
    return preprocess_verse(verse).split()[-1]

def verse_rhyme(verse):
    """Gets the rhyme of a verse"""
    # Get the last word
    w = last_word(verse)
    # Get the last two phones
    phones = pr.phones_for_word(w)
    if phones:
        last = re.match('.+ ([a-zA-Z]+[0-9][^0-9]*)$', phones[0])
        if last:
            rhyme = last.group(1)
            return rhyme
        else:
            return phones[0]
    else:
        return ""



def generate_line(markov_model, max_syllables=11):
    line = []
    current_word = random.choice(markov_model["__start__"])
    line.append(current_word)
    syllable_count = count_syllables(current_word)

    while syllable_count < max_syllables:
        
        if current_word == "__end__":
            break

        next_words = markov_model.get(current_word, {"next_word": ["__end__"], "probabilities": [1.0]})
        
        valid_next_words = [word for word, prob in zip(next_words["next_word"], next_words["probabilities"]) if syllable_count + count_syllables(word) <= max_syllables]
        
        if not valid_next_words:
            break
        
        
        
        next_word = random.choice(valid_next_words)

        while next_word == "__end__":
            current_word = random.choice(markov_model["__start__"])
            next_words = markov_model.get(current_word, {"next_word": ["__end__"], "probabilities": [1.0]})
            valid_next_words = [word for word, prob in zip(next_words["next_word"], next_words["probabilities"]) if syllable_count + count_syllables(word) <= max_syllables]
            if not valid_next_words:
                break
            next_word = random.choice(valid_next_words)

        
        line.append(next_word)
        current_word = next_word
        syllable_count += count_syllables(next_word)

    return " ".join(line[:-1])


# # Function to generate a poem with multiple lines
def generate_poem(markov_model, line_count=14):
    poem = []
    for _ in range(line_count):
        line = generate_line(markov_model)
        poem.append(line)

    return "\n".join(poem)

def generate_poem(markov_model, line_count=14, max_syllables=11):
    poem = []
    for _ in range(line_count):
        line = generate_line(markov_model)
        poem.append(line)

    return "\n".join(poem)


# Function to build a Markov chain model from a list of lines with probabilities
def build_markov_model(lines):
    markov_model = {"__start__": [], "__end__": []}

    for line in lines:
        words = line.split()
        markov_model["__start__"].append(words[0])
        markov_model["__end__"].append(words[-1])

        for i in range(len(words) - 1):
            current_word = words[i]
            next_word = words[i + 1]

            if current_word not in markov_model:
                markov_model[current_word] = {"next_word": [], "probabilities": []}

            markov_model[current_word]["next_word"].append(next_word)

    for word in markov_model:
        if word not in ["__start__", "__end__"]:
            next_words = markov_model[word]["next_word"]
            word_count = len(next_words)
            next_word_set = list(set(next_words))  # Remove duplicates
            probabilities = [next_words.count(word) / word_count for word in next_word_set]
            markov_model[word]["next_word"] = next_word_set
            markov_model[word]["probabilities"] = probabilities

    return markov_model


# Build a Markov chain model from the list of lines
markov_model = build_markov_model(poem_lines)

# Generate a new poem with multiple lines
new_poem = generate_poem(markov_model, line_count=14)

print("Generated Poem:")
print(new_poem)

Generated Poem:
Let him when my vows are sweetest bud. unswept stone,
Can make faults, and owners of one hour
Of faults thy humour doth well beseem thy face should
To show it was builded far the fragrant rose, in
That heavy sleep a joy above that one
And ruin'd choirs, where your trespass now
But then ten times happy are restor'd and tombs
That sometimes anger thrusts into my
And almost thence this title do forgive
Save breed, to ruinate vowing new pay as they grew?
And simple truth miscall'd simplicity, world
But beauty's veil doth she quenched in thee a
Save what conscience is partly blind, be blessed
For it were to wet a look,
